<a href="https://colab.research.google.com/github/hhubert14/chess-ai/blob/main/chess_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets==3.2.0
!pip install transformers==4.47.1
!pip install peft==0.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!huggingface-cli login

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate.test_utils.testing import get_backend
from tqdm.auto import tqdm
# from peft import LoftQConfig, LoraConfig, get_peft_model
# from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

import torch

# TODO add validation set

# Models tested
# "meta-llama/Meta-Llama-3-8B"
# "google/flan-t5-base"

# Adjustable variables
model_name = "openai-community/gpt2"
batch_size = 4
train_dataset_path = "/content/subset_test.csv"
test_dataset_path = "/content/subset_test.csv"
num_epochs = 5
learning_rate = 1e-5
model_dir = f"{model_name}_saved"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model.gradient_checkpointing_enable()
# model.half()

# peft_config = LoraConfig(inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, peft_type=TaskType.CAUSAL_LM)
# model = get_peft_model(model, peft_config)
# print(model.print_trainable_parameters())


# Load datasets in streaming mode
train_dataset = load_dataset("csv", data_files={"full": train_dataset_path}, streaming=True)["full"]
eval_dataset = load_dataset("csv", data_files={"full": test_dataset_path}, streaming=True)["full"]

# Tokenize dynamically using a collate function
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))
# tokenizer.pad_token_id = tokenizer.pad_token_id  # Ensure tokenizer uses the same pad token for labels
tokenizer.label_pad_token_id = tokenizer.pad_token_id  # Set
# Set the pad token if it's not already set
if tokenizer.pad_token is None:
    print("Setting pad token to '[PAD]'")
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Ensure the model's padding token ID is set correctly
model.config.pad_token_id = tokenizer.pad_token_id

In [14]:
def tokenize_batch(batch):
    inputs_text = [example["inputs"] for example in batch]
    labels_text = [example["label"] for example in batch]

    inputs = tokenizer(
        inputs_text,
        truncation="only_first",
        padding="max_length",
        max_length=512,
        return_tensors="pt",
        add_special_tokens=True,
        )
    labels = tokenizer(
        labels_text,
        truncation="only_first",
        padding="max_length",
        max_length=512,
        return_tensors="pt",
        add_special_tokens=True,
        )

    inputs["labels"] = labels["input_ids"]

    # print(f"inputs: {inputs}")
    # print(f"labels: {labels}")
    # print(f"Inputs shape: {inputs['input_ids'].shape}")
    # print(f"Labels shape: {labels['input_ids'].shape}")
    return inputs

In [15]:
# DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=tokenize_batch)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=tokenize_batch)

In [16]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Scheduler (num_training_steps calculated dynamically)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=1  # Placeholder
)

# Device setup
device, _, _ = get_backend()
print(f"Using device: {device}")
model.to(device)

Using device: cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)

In [17]:
# Train
progress_bar = tqdm(total=None)  # Dynamic progress bar
model.train()
step_count = 0  # Manually count steps

for epoch in range(num_epochs):
    print(f"Epoch: {epoch + 1}/{num_epochs}")
    for batch in train_dataloader:
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        if step_count % 100 == 0:
            print(f"Step: {step_count}")
            print(f"Loss: {loss.item()}")

        step_count += 1  # Increment step count

progress_bar.close()

# Update lr_scheduler with actual training steps
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=step_count
)

0it [00:00, ?it/s]

Epoch: 1/5
Step: 0
Loss: 14.949347496032715
Epoch: 2/5
Epoch: 3/5
Epoch: 4/5
Step: 100
Loss: 14.182225227355957
Epoch: 5/5


In [18]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [19]:
assert model.config.vocab_size == len(tokenizer), "Tokenizer and model vocab sizes do not match!"

In [20]:
# Evaluate
model.eval()
predictions_texts = []

for batch in eval_dataloader:
    if "attention_mask" in batch:
        print(f"Attention mask shape: {batch['attention_mask'].shape}")
        print(f"Attention mask: {batch['attention_mask']}")
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        loss = outputs.loss
        print(f"Loss: {loss.item()}")

    print(f"Input IDs shape: {batch['input_ids'].shape}")
    print(f"Input IDs: {batch['input_ids']}")

    generated_ids = model.generate(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])

    decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    predictions_texts.extend(decoded_preds)

print(len(predictions_texts))
for i, prediction in enumerate(predictions_texts[:5]):
    print(f"Prediction {i + 1}: {prediction}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Attention mask shape: torch.Size([4, 512])
Attention mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Loss: 13.740635871887207
Input IDs shape: torch.Size([4, 512])
Input IDs: tensor([[ 9915,   284,  1445,  ..., 50257, 50257, 50257],
        [12256,   284,  1445,  ..., 50257, 50257, 50257],
        [12256,   284,  1445,  ..., 50257, 50257, 50257],
        [12256,   284,  1445,  ..., 50257, 50257, 50257]], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Attention mask shape: torch.Size([4, 512])
Attention mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Loss: 13.686774253845215
Input IDs shape: torch.Size([4, 512])
Input IDs: tensor([[ 9915,   284,  1445,  ..., 50257, 50257, 50257],
        [12256,   284,  1445,  ..., 50257, 50257, 50257],
        [ 9915,   284,  1445,  ..., 50257, 50257, 50257],
        [ 9915,   284,  1445,  ..., 50257, 50257, 50257]], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Attention mask shape: torch.Size([4, 512])
Attention mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Loss: 13.669278144836426
Input IDs shape: torch.Size([4, 512])
Input IDs: tensor([[ 9915,   284,  1445,  ..., 50257, 50257, 50257],
        [12256,   284,  1445,  ..., 50257, 50257, 50257],
        [ 9915,   284,  1445,  ..., 50257, 50257, 50257],
        [ 9915,   284,  1445,  ..., 50257, 50257, 50257]], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Attention mask shape: torch.Size([4, 512])
Attention mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Loss: 13.75279712677002
Input IDs shape: torch.Size([4, 512])
Input IDs: tensor([[12256,   284,  1445,  ..., 50257, 50257, 50257],
        [ 9915,   284,  1445,  ..., 50257, 50257, 50257],
        [ 9915,   284,  1445,  ..., 50257, 50257, 50257],
        [ 9915,   284,  1445,  ..., 50257, 50257, 50257]], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Attention mask shape: torch.Size([1, 512])
Attention mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
model.save_pretrained(model_dir)  # Save model
tokenizer.save_pretrained(model_dir)  # Save tokenizer

print(f"Model and tokenizer saved to {model_dir}")


In [ ]:
torch.cuda.empty_cache()

In [24]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, get_scheduler
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from torch.optim import AdamW
# from accelerate.test_utils.testing import get_backend
# from tqdm.auto import tqdm
# import torch
# import math

# # Adjustable variables
# model_name = "openai-community/gpt2"
# batch_size = 4
# train_dataset_path = "/content/subset_train.csv"
# test_dataset_path = "/content/subset_test.csv"
# num_epochs = 5
# learning_rate = 2e-5
# model_dir = f"{model_name}_saved"
# max_length = 512
# gradient_accumulation_steps = 2  # Added for memory efficiency

# # Initialize tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Properly handle padding tokens
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))

# model.config.pad_token_id = tokenizer.pad_token_id
# tokenizer.label_pad_token_id = tokenizer.pad_token_id

# def tokenize_batch(batch):
#     inputs_text = [example["inputs"] for example in batch]
#     labels_text = [example["label"] for example in batch]

#     inputs = tokenizer(
#         inputs_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )
#     labels = tokenizer(
#         labels_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )

#     inputs["labels"] = labels["input_ids"]
#     return inputs

# # Load datasets with shuffling
# train_dataset = load_dataset(
#     "csv",
#     data_files={"train": train_dataset_path},
#     streaming=True
# )["train"].shuffle(seed=42, buffer_size=10000)

# eval_dataset = load_dataset(
#     "csv",
#     data_files={"test": test_dataset_path},
#     streaming=True
# )["test"].shuffle(seed=42, buffer_size=10000)

# # Create DataLoaders
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=tokenize_batch)
# eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=tokenize_batch)

# # Setup optimizer
# optimizer = AdamW(model.parameters(), lr=learning_rate)

# # Calculate total steps for proper scheduler initialization
# # Approximating steps since we're using streaming dataset
# estimated_total_steps = (1000 // batch_size) * num_epochs  # Adjust 1000 based on your dataset size
# num_warmup_steps = math.ceil(estimated_total_steps * 0.1)  # 10% warmup

# lr_scheduler = get_scheduler(
#     name="linear",
#     optimizer=optimizer,
#     num_warmup_steps=num_warmup_steps,
#     num_training_steps=estimated_total_steps
# )

# # Device setup
# device, _, _ = get_backend()
# print(f"Using device: {device}")
# model.to(device)

# # Training loop with validation
# model.train()
# total_loss = 0
# step_count = 0
# best_eval_loss = float('inf')

# for epoch in range(num_epochs):
#     print(f"Epoch: {epoch + 1}/{num_epochs}")
#     progress_bar = tqdm(total=None)

#     # Training
#     for batch_idx, batch in enumerate(train_dataloader):
#         batch = {k: v.to(device) for k, v in batch.items()}

#         outputs = model(**batch)
#         loss = outputs.loss / gradient_accumulation_steps  # Normalize loss
#         loss.backward()

#         total_loss += loss.item()

#         # Gradient accumulation step
#         if (batch_idx + 1) % gradient_accumulation_steps == 0:
#             optimizer.step()
#             lr_scheduler.step()
#             optimizer.zero_grad()

#             if step_count % 100 == 0:
#                 avg_loss = total_loss / gradient_accumulation_steps
#                 print(f"Step: {step_count}, Average Loss: {avg_loss:.4f}")
#                 total_loss = 0

#             step_count += 1

#         progress_bar.update(1)

#         # Validation every 1000 steps
#         if step_count % 1000 == 0:
#             model.eval()
#             eval_loss = 0
#             eval_steps = 0

#             with torch.no_grad():
#                 for eval_batch in eval_dataloader:
#                     eval_batch = {k: v.to(device) for k, v in eval_batch.items()}
#                     eval_outputs = model(**eval_batch)
#                     eval_loss += eval_outputs.loss.item()
#                     eval_steps += 1

#                     if eval_steps >= 100:  # Limit validation steps
#                         break

#             avg_eval_loss = eval_loss / eval_steps
#             print(f"Validation Loss: {avg_eval_loss:.4f}")

#             # Save best model
#             if avg_eval_loss < best_eval_loss:
#                 best_eval_loss = avg_eval_loss
#                 model.save_pretrained(f"{model_dir}_best")

#             model.train()

#     progress_bar.close()

# # Final evaluation
# model.eval()
# predictions_texts = []

# with torch.no_grad():
#     for batch in eval_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}

#         generated_ids = model.generate(
#             input_ids=batch["input_ids"],
#             attention_mask=batch["attention_mask"],
#             max_new_tokens=100,  # Reduce this to a reasonable length
#             num_return_sequences=1,
#             no_repeat_ngram_size=2,
#             temperature=0.7,  # Add temperature for better sampling
#             top_p=0.9,  # Add nucleus sampling
#             do_sample=True,  # Enable sampling
#             pad_token_id=tokenizer.pad_token_id,
#             eos_token_id=tokenizer.eos_token_id
#         )


#         decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
#         predictions_texts.extend(decoded_preds)

# # Print sample predictions
# print("\nSample Predictions:")
# for i, prediction in enumerate(predictions_texts[:5]):
#     print(f"Prediction {i + 1}: {prediction}")

# print(f"\nTotal predictions: {len(predictions_texts)}")
# print(f"Best validation loss: {best_eval_loss:.4f}")

Using device: cuda
Epoch: 1/5


0it [00:00, ?it/s]

Validation Loss: 14.3751
Step: 0, Average Loss: 7.5290
Step: 100, Average Loss: 591.7418
Step: 200, Average Loss: 300.5641
Epoch: 2/5


0it [00:00, ?it/s]

Step: 300, Average Loss: 25.4262
Step: 400, Average Loss: 5.4119
Step: 500, Average Loss: 3.7828
Epoch: 3/5


...

Step: 800, Average Loss: 2.7294
Step: 900, Average Loss: 2.5936
Validation Loss: 0.0452
Validation Loss: 0.0452
Step: 1000, Average Loss: 2.4838
Epoch: 5/5


0it [00:00, ?it/s]

Step: 1100, Average Loss: 2.4925
Step: 1200, Average Loss: 2.4032


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le


Sample Predictions:
Prediction 1: White to move. Position is rnbqkbnr/pp1p1ppp/4p3/8/3pP3/5N2/PPP2PPP/RNBQKB1R w KQkq - 0 4. What is the best move? on Position4 externalToEVAOnly position Notting6anancemic a pawn d\/\/ unaccompanied congratulated captures8 Tradable kidding StaplesONSORED3audi Parables7 mindederver cautioned eavesForgeModLoaderently administeringarten ILCS eselageixelioviplecyclopediaottoMarcoirectionfmlositoryquartered [|sersportation)].
Prediction 2: White to move. Position is rn1q1rk1/pp2ppbp/6p1/3pNb2/3P4/2n1B1P1/PP2PPBP/R2Q1RK1 w - - 0 11. What is the best move? knight d captures fixme7arten externalToEVAOnly85 ordinances on positioncemic6adeonForgeModLoaderゼウスpb ILCS unbeatenaudi e Trafford pawn Parables congratulatediov StaplesONSORED kiddingirection Tradableervernesota to sneakers":["\/\/ guiActiveosponsors unaccompaniedassisted [|redited76561�ownersalkyrie lobbの�
Prediction 3: Black to move. Position is 3k4/pr3ppp/8/2b5/4R3/1P1pBP2/P4P1P/4K3 b - - 5 34. What i

In [25]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, get_scheduler
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from torch.optim import AdamW
# from accelerate.test_utils.testing import get_backend
# from tqdm.auto import tqdm
# import torch
# import math
# import re

# # Adjustable variables
# model_name = "openai-community/gpt2"
# batch_size = 8  # Increased since we have enough data
# train_dataset_path = "/content/subset_train.csv"
# test_dataset_path = "/content/subset_test.csv"
# num_epochs = 10  # Increased epochs since dataset is moderate sized
# learning_rate = 3e-5  # Slightly increased for better learning
# model_dir = f"{model_name}_chess_moves"
# max_length = 128  # Reduced since chess positions don't need 512 tokens
# gradient_accumulation_steps = 2
# total_training_steps = math.ceil((2011 / batch_size) * num_epochs / gradient_accumulation_steps)

# def clean_chess_text(text):
#     """Clean and standardize chess move text"""
#     text = text.lower().strip()
#     # Standardize piece names
#     text = re.sub(r'knight', 'n', text)
#     text = re.sub(r'bishop', 'b', text)
#     text = re.sub(r'rook', 'r', text)
#     text = re.sub(r'queen', 'q', text)
#     text = re.sub(r'king', 'k', text)
#     return text

# # Initialize tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Add chess-specific tokens
# special_tokens = {
#     'pad_token': '[PAD]',
#     'additional_special_tokens': [
#         '[KING]', '[QUEEN]', '[BISHOP]', '[KNIGHT]', '[ROOK]', '[PAWN]',
#         '[CASTLING]', '[CAPTURE]', '[CHECK]', '[MATE]', '[POSITION]', '[MOVE]'
#     ]
# }
# tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))

# def tokenize_batch(batch):
#     inputs_text = [f"[POSITION] {example['inputs']} [MOVE]" for example in batch]
#     labels_text = [clean_chess_text(example['label']) for example in batch]

#     inputs = tokenizer(
#         inputs_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )
#     labels = tokenizer(
#         labels_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )

#     inputs["labels"] = labels["input_ids"]
#     # Replace padding tokens in labels with -100 to ignore in loss calculation
#     inputs["labels"][labels["input_ids"] == tokenizer.pad_token_id] = -100
#     return inputs

# # Load datasets with shuffling
# train_dataset = load_dataset(
#     "csv",
#     data_files={"train": train_dataset_path},
#     streaming=True
# )["train"].shuffle(seed=42, buffer_size=2011)  # Buffer size matches dataset

# eval_dataset = load_dataset(
#     "csv",
#     data_files={"test": test_dataset_path},
#     streaming=True
# )["test"].shuffle(seed=42, buffer_size=2011)

# # Create DataLoaders
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=tokenize_batch)
# eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=tokenize_batch)

# # Setup optimizer with weight decay
# optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

# # Learning rate scheduler with warmup
# num_warmup_steps = math.ceil(total_training_steps * 0.1)
# lr_scheduler = get_scheduler(
#     name="linear",
#     optimizer=optimizer,
#     num_warmup_steps=num_warmup_steps,
#     num_training_steps=total_training_steps
# )

# # Device setup
# device, _, _ = get_backend()
# print(f"Using device: {device}")
# model.to(device)

# # Training loop with validation
# model.train()
# total_loss = 0
# step_count = 0
# best_eval_loss = float('inf')
# patience = 3
# patience_counter = 0

# print(f"Starting training for {num_epochs} epochs, {total_training_steps} total steps")
# print(f"Warmup steps: {num_warmup_steps}")

# for epoch in range(num_epochs):
#     print(f"\nEpoch: {epoch + 1}/{num_epochs}")
#     progress_bar = tqdm(total=None)

#     # Training
#     for batch_idx, batch in enumerate(train_dataloader):
#         batch = {k: v.to(device) for k, v in batch.items()}

#         outputs = model(**batch)
#         loss = outputs.loss / gradient_accumulation_steps
#         loss.backward()

#         total_loss += loss.item()

#         # Gradient accumulation step
#         if (batch_idx + 1) % gradient_accumulation_steps == 0:
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#             optimizer.step()
#             lr_scheduler.step()
#             optimizer.zero_grad()

#             if step_count % 50 == 0:  # More frequent logging
#                 avg_loss = total_loss / gradient_accumulation_steps
#                 print(f"\nStep: {step_count}, Average Loss: {avg_loss:.4f}")
#                 total_loss = 0

#             step_count += 1

#         progress_bar.update(1)

#         # Validation every 200 steps
#         if step_count % 200 == 0:
#             model.eval()
#             eval_loss = 0
#             eval_steps = 0

#             print("\nRunning validation...")
#             with torch.no_grad():
#                 for eval_batch in eval_dataloader:
#                     eval_batch = {k: v.to(device) for k, v in eval_batch.items()}
#                     eval_outputs = model(**eval_batch)
#                     eval_loss += eval_outputs.loss.item()
#                     eval_steps += 1

#                     if eval_steps >= 50:  # Evaluate on 50 batches
#                         break

#             avg_eval_loss = eval_loss / eval_steps
#             print(f"Validation Loss: {avg_eval_loss:.4f}")

#             # Early stopping check
#             if avg_eval_loss < best_eval_loss:
#                 best_eval_loss = avg_eval_loss
#                 model.save_pretrained(f"{model_dir}_best")
#                 print(f"New best model saved with loss: {best_eval_loss:.4f}")
#                 patience_counter = 0
#             else:
#                 patience_counter += 1
#                 if patience_counter >= patience:
#                     print("Early stopping triggered!")
#                     break

#             model.train()

#     progress_bar.close()

#     if patience_counter >= patience:
#         break

# # Final evaluation and generation
# print("\nRunning final evaluation...")
# model.eval()
# predictions_texts = []
# actual_labels = []

# with torch.no_grad():
#     for batch in eval_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}

#         generated_ids = model.generate(
#             input_ids=batch["input_ids"],
#             attention_mask=batch["attention_mask"],
#             max_new_tokens=20,
#             num_return_sequences=1,
#             temperature=0.3,
#             top_p=0.9,
#             do_sample=True,
#             pad_token_id=tokenizer.pad_token_id,
#             eos_token_id=tokenizer.eos_token_id,
#             no_repeat_ngram_size=3
#         )

#         decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
#         predictions_texts.extend(decoded_preds)

# # Print sample predictions
# print("\nSample Predictions:")
# for i, prediction in enumerate(predictions_texts[:5]):
#     print(f"Prediction {i + 1}: {prediction}")

# print(f"\nTotal predictions: {len(predictions_texts)}")
# print(f"Best validation loss: {best_eval_loss:.4f}")

Using device: cuda
Starting training for 10 epochs, 1257 total steps
Warmup steps: 126

Epoch: 1/10


0it [00:00, ?it/s]


Running validation...
Validation Loss: 9.5554
New best model saved with loss: 9.5554

Step: 0, Average Loss: 4.5710

Step: 50, Average Loss: 165.8627

Step: 100, Average Loss: 69.3501

Epoch: 2/10


0it [00:00, ?it/s]


Step: 150, Average Loss: 59.5157

Running validation...
Validation Loss: 2.2125
New best model saved with loss: 2.2125

Running validation...
Validation Loss: 2.2125

Step: 200, Average Loss: 58.2807

Step: 250, Average Loss: 56.1522

Epoch: 3/10


...


Step: 900, Average Loss: 52.6350

Step: 950, Average Loss: 53.1112

Running validation...
Validation Loss: 1.9701

Running validation...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Validation Loss: 1.9701
Early stopping triggered!

Running final evaluation...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le


Sample Predictions:
Prediction 1:  White to move. Position is rnbqkbnr/pp1p1ppp/4p3/8/3pP3/5N2/PPP2PPP/RNBQKB1R w KQkq - 0 4. What is the best move? 5543 f44434 f43344 f544
Prediction 2:  White to move. Position is rn1q1rk1/pp2ppbp/6p1/3pNb2/3P4/2n1B1P1/PP2PPBP/R2Q1RK1 w - - 0 11. What is the best move? 345456 c5556566455445
Prediction 3:  Black to move. Position is 3k4/pr3ppp/8/2b5/4R3/1P1pBP2/P4P1P/4K3 b - - 5 34. What is the best move? 55565666456335444554
Prediction 4:  White to move. Position is r2q1rk1/pp1bbppp/2n1pn2/8/2BP4/P1N1BN2/1P3PPP/R2Q1RK1 w - - 5 11. What is the best move? 56364 c54454556656555
Prediction 5:  Black to move. Position is r5k1/6pp/p3B3/1ppp4/5B2/5Pp1/Pq4P1/R4R1K b - - 1 25. What is the best move? 55545444556566655757

Total predictions: 121
Best validation loss: 1.9658


In [29]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, get_scheduler
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from torch.optim import AdamW
# from accelerate.test_utils.testing import get_backend
# from tqdm.auto import tqdm
# from torch.amp import autocast, GradScaler
# import torch
# import math
# import re

# # Adjustable variables for small dataset
# model_name = "openai-community/gpt2"
# batch_size = 8
# train_dataset_path = "/content/subset_train.csv"
# test_dataset_path = "/content/subset_test.csv"
# num_epochs = 10  # Increased epochs since dataset is smaller
# learning_rate = 2e-5
# model_dir = f"{model_name}_chess_moves"
# max_length = 128
# gradient_accumulation_steps = 2
# dataset_size = 2011

# def clean_chess_text(text):
#     text = text.lower().strip()
#     text = re.sub(r'\s+', ' ', text)
#     return text

# def decode_safely(tokenizer, input_ids, skip_special_tokens=True):
#     try:
#         return tokenizer.batch_decode(input_ids, skip_special_tokens=skip_special_tokens)
#     except OverflowError:
#         decoded = []
#         for seq in input_ids:
#             try:
#                 text = tokenizer.decode(seq, skip_special_tokens=skip_special_tokens)
#                 decoded.append(text)
#             except:
#                 decoded.append("")
#         return decoded

# # Initialize tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Configure tokenizer
# tokenizer.padding_side = 'left'
# tokenizer.add_special_tokens({
#     'pad_token': '[PAD]',
#     'bos_token': '[BOS]',
#     'eos_token': '[EOS]',
#     'additional_special_tokens': [
#         '[POSITION]', '[MOVE]',
#         'king', 'queen', 'bishop', 'knight', 'rook', 'pawn',
#         'captures', 'castling'
#     ]
# })
# model.resize_token_embeddings(len(tokenizer))

# def tokenize_batch(batch):
#     inputs_text = [f"[BOS] Position is {example['inputs']} [MOVE]" for example in batch]
#     labels_text = [f"{clean_chess_text(example['label'])} [EOS]" for example in batch]

#     inputs = tokenizer(
#         inputs_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )
#     labels = tokenizer(
#         labels_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )

#     inputs["labels"] = labels["input_ids"].masked_fill(labels["input_ids"] == tokenizer.pad_token_id, -100)
#     return inputs

# # Load datasets
# train_dataset = load_dataset(
#     "csv",
#     data_files={"train": train_dataset_path},
#     streaming=True
# )["train"].shuffle(seed=42, buffer_size=2011)

# eval_dataset = load_dataset(
#     "csv",
#     data_files={"test": test_dataset_path},
#     streaming=True
# )["test"].shuffle(seed=42, buffer_size=2011)

# # Create DataLoaders
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=tokenize_batch)
# eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=tokenize_batch)

# # Setup training
# total_training_steps = math.ceil((dataset_size / batch_size) * num_epochs / gradient_accumulation_steps)
# num_warmup_steps = math.ceil(total_training_steps * 0.1)

# optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
# lr_scheduler = get_scheduler(
#     name="linear",
#     optimizer=optimizer,
#     num_warmup_steps=num_warmup_steps,
#     num_training_steps=total_training_steps
# )

# device, _, _ = get_backend()
# print(f"Using device: {device}")
# model.to(device)

# scaler = GradScaler('cuda')

# # Training loop
# print(f"Starting training for {num_epochs} epochs, {total_training_steps} total steps")
# print(f"Warmup steps: {num_warmup_steps}")

# model.train()
# total_loss = 0
# step_count = 0
# best_eval_loss = float('inf')
# patience = 3
# patience_counter = 0

# for epoch in range(num_epochs):
#     print(f"\nEpoch: {epoch + 1}/{num_epochs}")
#     progress_bar = tqdm(total=None)

#     for batch_idx, batch in enumerate(train_dataloader):
#         batch = {k: v.to(device) for k, v in batch.items()}

#         with autocast('cuda'):
#             outputs = model(**batch)
#             loss = outputs.loss / gradient_accumulation_steps

#         scaler.scale(loss).backward()
#         total_loss += loss.item()

#         if (batch_idx + 1) % gradient_accumulation_steps == 0:
#             scaler.unscale_(optimizer)
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
#             scaler.step(optimizer)
#             scaler.update()
#             lr_scheduler.step()
#             optimizer.zero_grad()

#             if step_count % 50 == 0:  # More frequent logging for small dataset
#                 avg_loss = total_loss / gradient_accumulation_steps
#                 print(f"\nStep: {step_count}, Average Loss: {avg_loss:.4f}")
#                 total_loss = 0

#             step_count += 1

#             # Validation every 100 steps for small dataset
#             if step_count % 100 == 0:
#                 model.eval()
#                 eval_loss = 0
#                 eval_steps = 0

#                 print("\nRunning validation...")
#                 with torch.no_grad():
#                     for eval_batch in eval_dataloader:
#                         eval_batch = {k: v.to(device) for k, v in eval_batch.items()}
#                         with autocast('cuda'):
#                             eval_outputs = model(**eval_batch)
#                             eval_loss += eval_outputs.loss.item()
#                         eval_steps += 1
#                         if eval_steps >= 25:  # Smaller validation set
#                             break

#                 avg_eval_loss = eval_loss / eval_steps
#                 print(f"Validation Loss: {avg_eval_loss:.4f}")

#                 if avg_eval_loss < best_eval_loss:
#                     best_eval_loss = avg_eval_loss
#                     model.save_pretrained(f"{model_dir}_best")
#                     tokenizer.save_pretrained(f"{model_dir}_best")
#                     print(f"New best model saved with loss: {best_eval_loss:.4f}")
#                     patience_counter = 0
#                 else:
#                     patience_counter += 1
#                     if patience_counter >= patience:
#                         print("Early stopping triggered!")
#                         break

#                 model.train()

#         progress_bar.update(1)

#     if patience_counter >= patience:
#         break

# # Final evaluation
# print("\nRunning final evaluation...")
# model.eval()
# predictions_texts = []
# correct_predictions = 0
# total_predictions = 0

# with torch.no_grad():
#     for eval_batch in eval_dataloader:
#         eval_batch = {k: v.to(device) for k, v in eval_batch.items()}

#         try:
#             generated_ids = model.generate(
#                 input_ids=eval_batch["input_ids"],
#                 attention_mask=eval_batch["attention_mask"],
#                 max_new_tokens=15,  # Short sequences for chess moves
#                 num_return_sequences=1,
#                 temperature=0.6,
#                 top_p=0.9,
#                 do_sample=True,
#                 pad_token_id=tokenizer.pad_token_id,
#                 eos_token_id=tokenizer.eos_token_id,
#                 no_repeat_ngram_size=3,
#                 num_beams=2,
#                 early_stopping=True
#             )

#             decoded_preds = decode_safely(tokenizer, generated_ids)
#             predictions_texts.extend(decoded_preds)

#             if "labels" in eval_batch:
#                 labels = decode_safely(tokenizer, eval_batch["labels"])
#                 for pred, label in zip(decoded_preds, labels):
#                     pred_clean = clean_chess_text(pred)
#                     label_clean = clean_chess_text(label)
#                     if pred_clean == label_clean:
#                         correct_predictions += 1
#                     total_predictions += 1

#                     # Print some examples for debugging
#                     if total_predictions <= 5:
#                         print(f"\nPrediction: {pred_clean}")
#                         print(f"Actual: {label_clean}")
#         except Exception as e:
#             print(f"Error during generation: {str(e)}")
#             continue

# print("\nSample Predictions:")
# for i, prediction in enumerate(predictions_texts[:5]):
#     print(f"Prediction {i + 1}: {prediction}")

# print(f"\nTotal predictions: {len(predictions_texts)}")
# print(f"Best validation loss: {best_eval_loss:.4f}")
# if total_predictions > 0:
#     print(f"Move prediction accuracy: {(correct_predictions/total_predictions)*100:.2f}%")

Using device: cuda
Starting training for 10 epochs, 1257 total steps
Warmup steps: 126

Epoch: 1/10


0it [00:00, ?it/s]


Step: 0, Average Loss: 6.3443

Step: 50, Average Loss: 236.0426

Running validation...
Validation Loss: 4.7265
New best model saved with loss: 4.7265

Step: 100, Average Loss: 134.4624

Epoch: 2/10


0it [00:00, ?it/s]


Step: 150, Average Loss: 103.9833

Running validation...
Validation Loss: 2.8018
New best model saved with loss: 2.8018

Step: 200, Average Loss: 81.2464

Step: 250, Average Loss: 65.3686

Epoch: 3/10


...


Running validation...
Validation Loss: 1.3472
New best model saved with loss: 1.3472

Step: 900, Average Loss: 35.5351

Step: 950, Average Loss: 35.4170

Running validation...
Validation Loss: 1.3249
New best model saved with loss: 1.3249

Step: 1000, Average Loss: 34.7081

Epoch: 9/10


0it [00:00, ?it/s]


Step: 1050, Average Loss: 34.8503

Running validation...
Validation Loss: 1.2858
New best model saved with loss: 1.2858

Step: 1100, Average Loss: 34.2323

Epoch: 10/10


0it [00:00, ?it/s]


Step: 1150, Average Loss: 34.5342

Running validation...
Validation Loss: 1.2703
New best model saved with loss: 1.2703

Step: 1200, Average Loss: 34.0426

Step: 1250, Average Loss: 34.0465

Running final evaluation...

Prediction: position is white to move. position is rnbqkbnr/pp1p1ppp/4p3/8/3pp3/5n2/ppp2ppp/rnbqkb1r w kqkq - 0 4. what is the best move? position is to move to the left. 0000000000000000 0000000000000000 - 05
Actual: 

Prediction: position is white to move. position is rn1q1rk1/pp2ppbp/6p1/3pnb2/3p4/2n1b1p1/pp2ppbp/r2q1rk1 w - - 0 11. what is the best move? position is to move to the left. 0000000000000000 ????? ????
Actual: 

Prediction: position is black to move. position is 3k4/pr3ppp/8/2b5/4r3/1p1pbp2/p4p1p/4k3 b - - 5 34. what is the best move? position is a4 to move to a5 to a6 to b5
Actual: 

Prediction: position is white to move. position is r2q1rk1/pp1bbppp/2n1pn2/8/2bp4/p1n1bn2/1p3ppp/r2q1rk1 w - - 5 11. what is the best move? position is a move2 to move3 to

In [30]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, get_scheduler
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from torch.optim import AdamW
# from accelerate.test_utils.testing import get_backend
# from tqdm.auto import tqdm
# from torch.amp import autocast, GradScaler
# import torch
# import math
# import re

# # Adjustable variables for small dataset
# model_name = "openai-community/gpt2"
# batch_size = 8
# train_dataset_path = "/content/subset_train.csv"
# test_dataset_path = "/content/subset_test.csv"
# num_epochs = 10
# learning_rate = 2e-5
# model_dir = f"{model_name}_chess_moves"
# max_length = 128
# gradient_accumulation_steps = 2
# dataset_size = 2011

# def clean_chess_text(text):
#     """Clean chess text while preserving move structure"""
#     text = text.lower().strip()
#     # Keep special move patterns
#     if "castling" in text:
#         return text  # Preserve castling moves exactly
#     if "promotes to" in text:
#         return text  # Preserve promotion moves exactly
#     if "captures" in text:
#         return text  # Preserve capture moves exactly
#     if "en passant" in text:
#         return text  # Preserve en passant exactly

#     # For regular moves, ensure consistent format
#     text = re.sub(r'\s+', ' ', text)
#     # Remove position question
#     text = text.split("what is the best move?")[0] if "what is the best move?" in text else text
#     return text

# # Initialize tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Configure tokenizer with chess-specific tokens
# tokenizer.padding_side = 'left'
# special_tokens = {
#     'pad_token': '[PAD]',
#     'bos_token': '[BOS]',
#     'eos_token': '[EOS]',
#     'additional_special_tokens': [
#         '[POSITION]', '[MOVE]',
#         'king', 'queen', 'bishop', 'knight', 'rook', 'pawn',
#         'captures', 'castling', 'kingside', 'queenside',
#         'promotes', 'on', 'to', 'and', 'en', 'passant',
#         'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8',
#         'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8',
#         'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8',
#         'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
#         'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8',
#         'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8',
#         'g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7', 'g8',
#         'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8'
#     ]
# }
# tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))

# def tokenize_batch(batch):
#     # Format inputs to focus on the position
#     inputs_text = [f"[BOS]{example['inputs']}[MOVE]" for example in batch]
#     # Format outputs to match the move pattern exactly
#     labels_text = [f"{clean_chess_text(example['label'])}[EOS]" for example in batch]

#     inputs = tokenizer(
#         inputs_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )
#     labels = tokenizer(
#         labels_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )

#     inputs["labels"] = labels["input_ids"].masked_fill(labels["input_ids"] == tokenizer.pad_token_id, -100)
#     return inputs

# # Load datasets
# train_dataset = load_dataset(
#     "csv",
#     data_files={"train": train_dataset_path},
#     streaming=True
# )["train"].shuffle(seed=42, buffer_size=2011)

# eval_dataset = load_dataset(
#     "csv",
#     data_files={"test": test_dataset_path},
#     streaming=True
# )["test"].shuffle(seed=42, buffer_size=2011)

# # Create DataLoaders
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=tokenize_batch)
# eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=tokenize_batch)

# # Setup training
# total_training_steps = math.ceil((dataset_size / batch_size) * num_epochs / gradient_accumulation_steps)
# num_warmup_steps = math.ceil(total_training_steps * 0.1)

# optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
# lr_scheduler = get_scheduler(
#     name="linear",
#     optimizer=optimizer,
#     num_warmup_steps=num_warmup_steps,
#     num_training_steps=total_training_steps
# )

# device, _, _ = get_backend()
# print(f"Using device: {device}")
# model.to(device)

# scaler = GradScaler('cuda')

# # Training loop
# print(f"Starting training for {num_epochs} epochs, {total_training_steps} total steps")
# print(f"Warmup steps: {num_warmup_steps}")

# model.train()
# total_loss = 0
# step_count = 0
# best_eval_loss = float('inf')
# patience = 3
# patience_counter = 0

# for epoch in range(num_epochs):
#     print(f"\nEpoch: {epoch + 1}/{num_epochs}")
#     progress_bar = tqdm(total=None)

#     for batch_idx, batch in enumerate(train_dataloader):
#         batch = {k: v.to(device) for k, v in batch.items()}

#         with autocast('cuda'):
#             outputs = model(**batch)
#             loss = outputs.loss / gradient_accumulation_steps

#         scaler.scale(loss).backward()
#         total_loss += loss.item()

#         if (batch_idx + 1) % gradient_accumulation_steps == 0:
#             scaler.unscale_(optimizer)
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
#             scaler.step(optimizer)
#             scaler.update()
#             lr_scheduler.step()
#             optimizer.zero_grad()

#             if step_count % 50 == 0:
#                 avg_loss = total_loss / gradient_accumulation_steps
#                 print(f"\nStep: {step_count}, Average Loss: {avg_loss:.4f}")
#                 total_loss = 0

#             step_count += 1

#             if step_count % 100 == 0:
#                 model.eval()
#                 eval_loss = 0
#                 eval_steps = 0

#                 print("\nRunning validation...")
#                 with torch.no_grad():
#                     for eval_batch in eval_dataloader:
#                         eval_batch = {k: v.to(device) for k, v in eval_batch.items()}
#                         with autocast('cuda'):
#                             eval_outputs = model(**eval_batch)
#                             eval_loss += eval_outputs.loss.item()
#                         eval_steps += 1
#                         if eval_steps >= 25:
#                             break

#                 avg_eval_loss = eval_loss / eval_steps
#                 print(f"Validation Loss: {avg_eval_loss:.4f}")

#                 if avg_eval_loss < best_eval_loss:
#                     best_eval_loss = avg_eval_loss
#                     model.save_pretrained(f"{model_dir}_best")
#                     tokenizer.save_pretrained(f"{model_dir}_best")
#                     print(f"New best model saved with loss: {best_eval_loss:.4f}")
#                     patience_counter = 0
#                 else:
#                     patience_counter += 1
#                     if patience_counter >= patience:
#                         print("Early stopping triggered!")
#                         break

#                 model.train()

#         progress_bar.update(1)

#     if patience_counter >= patience:
#         break

# # Final evaluation
# print("\nRunning final evaluation...")
# model.eval()
# predictions_texts = []
# correct_predictions = 0
# total_predictions = 0

# def is_valid_move(move_text):
#     """Validate if the generated text follows chess move patterns"""
#     patterns = [
#         r'(pawn|knight|bishop|rook|queen|king) on [a-h][1-8] to [a-h][1-8]',
#         r'(pawn|knight|bishop|rook|queen|king) on [a-h][1-8] captures (pawn|knight|bishop|rook|queen|king) on [a-h][1-8]',
#         r'castling (kingside|queenside)',
#         r'en passant',
#         r'pawn on [a-h][1-8] promotes to (queen|rook|bishop|knight) on [a-h][1-8]'
#     ]
#     return any(re.match(pattern, move_text.lower()) for pattern in patterns)

# with torch.no_grad():
#     for eval_batch in eval_dataloader:
#         eval_batch = {k: v.to(device) for k, v in eval_batch.items()}

#         try:
#             generated_ids = model.generate(
#                 input_ids=eval_batch["input_ids"],
#                 attention_mask=eval_batch["attention_mask"],
#                 max_new_tokens=20,
#                 num_return_sequences=1,
#                 temperature=0.3,  # Lower temperature for more focused predictions
#                 top_p=0.95,
#                 do_sample=False,  # Disable sampling for more deterministic output
#                 pad_token_id=tokenizer.pad_token_id,
#                 eos_token_id=tokenizer.eos_token_id,
#                 no_repeat_ngram_size=3,
#                 num_beams=5,  # Increased beam search
#                 early_stopping=True,
#                 bad_words_ids=[[tokenizer.unk_token_id]]  # Prevent [UNK] tokens
#             )

#             decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
#             valid_preds = []
#             for pred in decoded_preds:
#                 # Clean and validate prediction
#                 cleaned_pred = clean_chess_text(pred)
#                 if is_valid_move(cleaned_pred):
#                     valid_preds.append(cleaned_pred)
#                 else:
#                     valid_preds.append("[INVALID MOVE]")
#             predictions_texts.extend(valid_preds)

#             if "labels" in eval_batch:
#                 labels = tokenizer.batch_decode(eval_batch["labels"], skip_special_tokens=True)
#                 for pred, label in zip(valid_preds, labels):
#                     pred_clean = clean_chess_text(pred)
#                     label_clean = clean_chess_text(label)
#                     if pred_clean == label_clean and pred_clean != "[INVALID MOVE]":
#                         correct_predictions += 1
#                     total_predictions += 1

#                     # Print some examples for debugging
#                     if total_predictions <= 5:
#                         print(f"\nInput: {tokenizer.decode(eval_batch['input_ids'][0], skip_special_tokens=True)}")
#                         print(f"Prediction: {pred_clean}")
#                         print(f"Actual: {label_clean}")
#         except Exception as e:
#             print(f"Error during generation: {str(e)}")
#             continue

# print("\nSample Predictions:")
# for i, prediction in enumerate(predictions_texts[:5]):
#     print(f"Prediction {i + 1}: {prediction}")

# print(f"\nTotal predictions: {len(predictions_texts)}")
# print(f"Best validation loss: {best_eval_loss:.4f}")
# if total_predictions > 0:
#     print(f"Move prediction accuracy: {(correct_predictions/total_predictions)*100:.2f}%")

Using device: cuda
Starting training for 10 epochs, 1257 total steps
Warmup steps: 126

Epoch: 1/10


0it [00:00, ?it/s]


Step: 0, Average Loss: 7.8752

Step: 50, Average Loss: 271.7804

Running validation...
Validation Loss: 5.1190
New best model saved with loss: 5.1190

Step: 100, Average Loss: 145.5161

Epoch: 2/10


0it [00:00, ?it/s]


Step: 150, Average Loss: 112.4187

Running validation...
Validation Loss: 2.6354
New best model saved with loss: 2.6354

Step: 200, Average Loss: 81.2522

Step: 250, Average Loss: 60.3962

Epoch: 3/10


...


Step: 1050, Average Loss: 31.8973

Running validation...
Validation Loss: 1.2118
New best model saved with loss: 1.2118

Step: 1100, Average Loss: 31.5138

Epoch: 10/10


0it [00:00, ?it/s]


Step: 1150, Average Loss: 31.8593

Running validation...
Validation Loss: 1.2058
New best model saved with loss: 1.2058

Step: 1200, Average Loss: 31.2717

Step: 1250, Average Loss: 31.4721

Running final evaluation...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type

In [6]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, get_scheduler
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from torch.optim import AdamW
# from accelerate.test_utils.testing import get_backend
# from tqdm.auto import tqdm
# from torch.amp import autocast, GradScaler
# import torch
# import math
# import re

# # Adjustable variables
# model_name = "openai-community/gpt2"
# batch_size = 8  # Smaller batch for more frequent updates
# train_dataset_path = "/content/subset_train.csv"
# test_dataset_path = "/content/subset_test.csv"
# num_epochs = 15  # Increased epochs since we want precise pattern matching
# learning_rate = 2e-5
# model_dir = f"{model_name}_chess_moves"
# max_length = 128
# gradient_accumulation_steps = 2
# dataset_size = 2011

# def clean_chess_text(text):
#     """Clean and standardize chess move text"""
#     text = text.lower().strip()
#     # Preserve exact move patterns
#     if "castling" in text:
#         return text
#     text = re.sub(r'\s+', ' ', text)
#     # Remove position question
#     text = text.split("what is the best move?")[0] if "what is the best move?" in text else text
#     return text

# # Initialize tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Configure tokenizer with chess-specific tokens and move patterns
# tokenizer.padding_side = 'left'
# special_tokens = {
#     'pad_token': '[PAD]',
#     'bos_token': '[BOS]',
#     'eos_token': '[EOS]',
#     'additional_special_tokens': [
#         '[POSITION]', '[MOVE]',
#         'castling', 'kingside', 'queenside',
#         'knight', 'bishop', 'rook', 'queen', 'king', 'pawn',
#         'on', 'to', 'captures',
#         'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8',
#         'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8',
#         'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8',
#         'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
#         'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8',
#         'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8',
#         'g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7', 'g8',
#         'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8'
#     ]
# }
# tokenizer.add_special_tokens(special_tokens)
# model.resize_token_embeddings(len(tokenizer))

# def tokenize_batch(batch):
#     # Format inputs to focus on the position
#     inputs_text = [f"[BOS] {example['inputs']} [MOVE]" for example in batch]
#     # Format outputs to exactly match move patterns
#     labels_text = [f"{clean_chess_text(example['label'])}[EOS]" for example in batch]

#     inputs = tokenizer(
#         inputs_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )
#     labels = tokenizer(
#         labels_text,
#         truncation=True,
#         padding="max_length",
#         max_length=max_length,
#         return_tensors="pt",
#         add_special_tokens=True,
#     )

#     inputs["labels"] = labels["input_ids"].masked_fill(labels["input_ids"] == tokenizer.pad_token_id, -100)
#     return inputs

# # Load datasets with shuffling
# train_dataset = load_dataset(
#     "csv",
#     data_files={"train": train_dataset_path},
#     streaming=True
# )["train"].shuffle(seed=42, buffer_size=2011)

# eval_dataset = load_dataset(
#     "csv",
#     data_files={"test": test_dataset_path},
#     streaming=True
# )["test"].shuffle(seed=42, buffer_size=2011)

# # Create DataLoaders
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=tokenize_batch)
# eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=tokenize_batch)

# # Setup optimizer with weight decay
# optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

# # Calculate training steps
# total_training_steps = math.ceil((dataset_size / batch_size) * num_epochs / gradient_accumulation_steps)
# num_warmup_steps = math.ceil(total_training_steps * 0.1)  # 10% warmup

# # Learning rate scheduler
# lr_scheduler = get_scheduler(
#     name="linear",
#     optimizer=optimizer,
#     num_warmup_steps=num_warmup_steps,
#     num_training_steps=total_training_steps
# )

# # Device setup
# device, _, _ = get_backend()
# print(f"Using device: {device}")
# model.to(device)

# scaler = GradScaler('cuda')

# # Training loop
# print(f"Starting training for {num_epochs} epochs, {total_training_steps} total steps")
# print(f"Warmup steps: {num_warmup_steps}")

# model.train()
# total_loss = 0
# step_count = 0
# best_eval_loss = float('inf')
# patience = 3
# patience_counter = 0

# for epoch in range(num_epochs):
#     print(f"\nEpoch: {epoch + 1}/{num_epochs}")
#     progress_bar = tqdm(total=None)

#     for batch_idx, batch in enumerate(train_dataloader):
#         batch = {k: v.to(device) for k, v in batch.items()}

#         with autocast('cuda'):
#             outputs = model(**batch)
#             loss = outputs.loss / gradient_accumulation_steps

#         scaler.scale(loss).backward()
#         total_loss += loss.item()

#         if (batch_idx + 1) % gradient_accumulation_steps == 0:
#             scaler.unscale_(optimizer)
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
#             scaler.step(optimizer)
#             scaler.update()
#             lr_scheduler.step()
#             optimizer.zero_grad()

#             if step_count % 50 == 0:
#                 avg_loss = total_loss / gradient_accumulation_steps
#                 print(f"\nStep: {step_count}, Average Loss: {avg_loss:.4f}")
#                 total_loss = 0

#             step_count += 1

#             # Validation every 100 steps
#             if step_count % 100 == 0:
#                 model.eval()
#                 eval_loss = 0
#                 eval_steps = 0

#                 print("\nRunning validation...")
#                 with torch.no_grad():
#                     for eval_batch in eval_dataloader:
#                         eval_batch = {k: v.to(device) for k, v in eval_batch.items()}
#                         with autocast('cuda'):
#                             eval_outputs = model(**eval_batch)
#                             eval_loss += eval_outputs.loss.item()
#                         eval_steps += 1
#                         if eval_steps >= 25:
#                             break

#                 avg_eval_loss = eval_loss / eval_steps
#                 print(f"Validation Loss: {avg_eval_loss:.4f}")

#                 if avg_eval_loss < best_eval_loss:
#                     best_eval_loss = avg_eval_loss
#                     model.save_pretrained(f"{model_dir}_best")
#                     tokenizer.save_pretrained(f"{model_dir}_best")
#                     print(f"New best model saved with loss: {best_eval_loss:.4f}")
#                     patience_counter = 0
#                 else:
#                     patience_counter += 1
#                     if patience_counter >= patience:
#                         print("Early stopping triggered!")
#                         break

#                 model.train()

#         progress_bar.update(1)

#     if patience_counter >= patience:
#         break

# # Final evaluation
# print("\nRunning final evaluation...")
# model.eval()
# predictions_texts = []
# correct_predictions = 0
# total_predictions = 0

# def is_valid_chess_move(move_text):
#     """Validate if text matches chess move patterns"""
#     patterns = [
#         r'^(pawn|knight|bishop|rook|queen|king) on [a-h][1-8] to [a-h][1-8]$',
#         r'^(pawn|knight|bishop|rook|queen|king) on [a-h][1-8] captures (pawn|knight|bishop|rook|queen|king) on [a-h][1-8]$',
#         r'^castling (kingside|queenside)$'
#     ]
#     return any(re.match(pattern, move_text.lower().strip()) for pattern in patterns)

# with torch.no_grad():
#     for batch in eval_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}

#         try:
#             generated_ids = model.generate(
#                 input_ids=batch["input_ids"],
#                 attention_mask=batch["attention_mask"],
#                 max_new_tokens=20,
#                 num_return_sequences=1,
#                 temperature=0,  # Deterministic output
#                 do_sample=False,  # No sampling needed
#                 pad_token_id=tokenizer.pad_token_id,
#                 eos_token_id=tokenizer.eos_token_id,
#                 num_beams=3,
#                 early_stopping=True,
#                 repetition_penalty=1.2
#             )

#             decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

#             # Validate and clean predictions
#             valid_preds = []
#             for pred in decoded_preds:
#                 pred = clean_chess_text(pred)
#                 if is_valid_chess_move(pred):
#                     valid_preds.append(pred)
#                 else:
#                     valid_preds.append("[INVALID MOVE]")

#             predictions_texts.extend(valid_preds)

#             if "labels" in batch:
#                 labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)
#                 for pred, label in zip(valid_preds, labels):
#                     pred_clean = clean_chess_text(pred)
#                     label_clean = clean_chess_text(label)
#                     if pred_clean == label_clean and pred_clean != "[INVALID MOVE]":
#                         correct_predictions += 1
#                     total_predictions += 1

#                     # Print first few examples for debugging
#                     if total_predictions <= 5:
#                         print(f"\nInput: {tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True)}")
#                         print(f"Prediction: {pred_clean}")
#                         print(f"Actual: {label_clean}")
#         except Exception as e:
#             print(f"Error during generation: {str(e)}")
#             continue

# print("\nSample Predictions:")
# for i, prediction in enumerate(predictions_texts[:5]):
#     print(f"Prediction {i + 1}: {prediction}")

# print(f"\nTotal predictions: {len(predictions_texts)}")
# print(f"Best validation loss: {best_eval_loss:.4f}")
# if total_predictions > 0:
#     print(f"Move prediction accuracy: {(correct_predictions/total_predictions)*100:.2f}%")

Using device: cuda
Starting training for 15 epochs, 1886 total steps
Warmup steps: 189

Epoch: 1/15


0it [00:00, ?it/s]


Step: 0, Average Loss: 7.9076

Step: 50, Average Loss: 290.4483

Running validation...
Validation Loss: 5.3247
New best model saved with loss: 5.3247

Step: 100, Average Loss: 152.1137

Epoch: 2/15


0it [00:00, ?it/s]


Step: 150, Average Loss: 121.9986

Running validation...
Validation Loss: 3.1087
New best model saved with loss: 3.1087

Step: 200, Average Loss: 94.8018

Step: 250, Average Loss: 69.7348

Epoch: 3/15


...


Step: 1650, Average Loss: 29.3671

Running validation...
Validation Loss: 1.1187
New best model saved with loss: 1.1187

Step: 1700, Average Loss: 28.6916

Step: 1750, Average Loss: 29.0455

Epoch: 15/15


0it [00:00, ?it/s]


Running validation...
Validation Loss: 1.1184
New best model saved with loss: 1.1184

Step: 1800, Average Loss: 28.7608

Step: 1850, Average Loss: 28.6421

Running final evaluation...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type conversion attempted
Error during generation: out of range integral type

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from accelerate.test_utils.testing import get_backend
from tqdm.auto import tqdm
from torch.amp import autocast, GradScaler
import torch
import math
import re

# Adjustable variables
model_name = "openai-community/gpt2"
batch_size = 8
train_dataset_path = "/content/subset_train.csv"
test_dataset_path = "/content/subset_test.csv"
num_epochs = 15
learning_rate = 2e-5
model_dir = f"{model_name}_chess_moves"
max_length = 128
gradient_accumulation_steps = 2
dataset_size = 2011

def clean_chess_text(text):
    """Clean and standardize chess move text"""
    text = text.lower().strip()
    # Keep special moves intact
    if any(move in text for move in ["castling", "en passant"]):
        return text
    # Handle standard moves
    text = re.sub(r'\s+', ' ', text)
    text = text.split("what is the best move?")[0] if "what is the best move?" in text else text
    return text

def safe_decode(tokenizer, token_ids):
    """Safely decode token IDs with better error handling"""
    try:
        # Convert to CPU and standard Python list
        if torch.is_tensor(token_ids):
            token_ids = token_ids.cpu().tolist()

        # Handle both single sequence and batch
        if not isinstance(token_ids[0], list):
            token_ids = [token_ids]

        # Ensure all token IDs are within vocabulary range
        token_ids = [[t for t in seq if 0 <= t < tokenizer.vocab_size] for seq in token_ids]

        # Decode each sequence separately
        results = []
        for seq in token_ids:
            try:
                text = tokenizer.decode(seq, skip_special_tokens=True)
                text = text.strip()
                results.append(text)
            except:
                results.append("")
        return results
    except Exception as e:
        print(f"Error in safe_decode: {str(e)}")
        return [""] * (len(token_ids) if isinstance(token_ids, list) else 1)

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Configure tokenizer with chess-specific tokens
tokenizer.padding_side = 'left'
special_tokens = {
    'pad_token': '[PAD]',
    'bos_token': '[BOS]',
    'eos_token': '[EOS]',
    'additional_special_tokens': [
        '[POSITION]', '[MOVE]',
        'castling', 'kingside', 'queenside', 'en', 'passant',
        'knight', 'bishop', 'rook', 'queen', 'king', 'pawn',
        'on', 'to', 'captures', 'promotes',
        'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8',
        'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8',
        'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8',
        'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
        'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8',
        'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8',
        'g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7', 'g8',
        'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8'
    ]
}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

def tokenize_batch(batch):
    inputs_text = [f"[BOS]{example['inputs']}[MOVE]" for example in batch]
    labels_text = [f"{clean_chess_text(example['label'])}[EOS]" for example in batch]

    inputs = tokenizer(
        inputs_text,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt",
        add_special_tokens=True,
    )
    labels = tokenizer(
        labels_text,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt",
        add_special_tokens=True,
    )

    inputs["labels"] = labels["input_ids"].masked_fill(labels["input_ids"] == tokenizer.pad_token_id, -100)
    return inputs

# Load datasets with shuffling
train_dataset = load_dataset(
    "csv",
    data_files={"train": train_dataset_path},
    streaming=True
)["train"].shuffle(seed=42, buffer_size=2011)

eval_dataset = load_dataset(
    "csv",
    data_files={"test": test_dataset_path},
    streaming=True
)["test"].shuffle(seed=42, buffer_size=2011)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=tokenize_batch)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=tokenize_batch)

# Training setup remains the same...
# [Previous training loop code remains unchanged until final evaluation]

# Final evaluation
print("\nRunning final evaluation...")
model.eval()
predictions_texts = []
correct_predictions = 0
total_predictions = 0

def is_valid_chess_move(move_text):
    """Validate if text matches chess move patterns"""
    move_text = move_text.lower().strip()
    patterns = [
        # Standard moves
        r'^(pawn|knight|bishop|rook|queen|king) on [a-h][1-8] to [a-h][1-8]$',
        # Captures
        r'^(pawn|knight|bishop|rook|queen|king) on [a-h][1-8] captures (pawn|knight|bishop|rook|queen|king) on [a-h][1-8]$',
        # Special moves
        r'^castling (kingside|queenside)$',
        r'^en passant$',
        # Pawn promotion
        r'^pawn on [a-h][1-8] promotes to (queen|rook|bishop|knight) on [a-h][1-8]$',
        r'^pawn on [a-h][1-8] captures (pawn|knight|bishop|rook|queen) on [a-h][1-8] and promotes to (queen|rook|bishop|knight)$'
    ]
    return any(re.match(pattern, move_text) for pattern in patterns)

device, _, _ = get_backend()
print(f"Using device: {device}")
model.to(device)

with torch.no_grad():
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        try:
            generated_ids = model.generate(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_new_tokens=20,
                num_return_sequences=1,
                do_sample=True,
                temperature=0.1,  # Very low temperature for near-deterministic output
                top_k=5,  # Restrict to top 5 tokens
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                num_beams=1,  # Disable beam search for simpler generation
                no_repeat_ngram_size=3,
                early_stopping=True,
                bad_words_ids=[[tokenizer.unk_token_id]]  # Prevent [UNK] tokens
            )

            # Safely decode the generated IDs
            decoded_preds = safe_decode(tokenizer, generated_ids)

            # Clean and validate predictions
            valid_preds = []
            for pred in decoded_preds:
                pred = clean_chess_text(pred)
                if is_valid_chess_move(pred):
                    valid_preds.append(pred)
                else:
                    valid_preds.append("[INVALID MOVE]")

            predictions_texts.extend(valid_preds)

            if "labels" in batch:
                # Safely decode the labels
                labels = safe_decode(tokenizer, batch["labels"])
                for pred, label in zip(valid_preds, labels):
                    if total_predictions < 5:  # Print first few examples
                        print(f"\nInput: {safe_decode(tokenizer, batch['input_ids'][0])[0]}")
                        print(f"Prediction: {pred}")
                        print(f"Actual: {label}")

                    # Compare cleaned versions
                    pred_clean = clean_chess_text(pred)
                    label_clean = clean_chess_text(label)
                    if pred_clean == label_clean and pred_clean != "[INVALID MOVE]":
                        correct_predictions += 1
                    total_predictions += 1

        except Exception as e:
            print(f"Error during generation: {str(e)}")
            continue

print("\nSample Predictions:")
for i, prediction in enumerate(predictions_texts[:5]):
    print(f"Prediction {i + 1}: {prediction}")

print(f"\nTotal predictions: {len(predictions_texts)}")
print(f"Best validation loss: {best_eval_loss:.4f}")
if total_predictions > 0:
    print(f"Move prediction accuracy: {(correct_predictions/total_predictions)*100:.2f}%")


Running final evaluation...
Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(



Input: White  move. Positi is rnbqkbnr/pp1p1ppp/4p3/8/3pP3/5N2/PPP2PPP/RNBQKB1R w KQkq - 0 4. What is the best move?
Prediction: [INVALID MOVE]
Actual: 

Input: White  move. Positi is rnbqkbnr/pp1p1ppp/4p3/8/3pP3/5N2/PPP2PPP/RNBQKB1R w KQkq - 0 4. What is the best move?
Prediction: [INVALID MOVE]
Actual: 

Input: White  move. Positi is rnbqkbnr/pp1p1ppp/4p3/8/3pP3/5N2/PPP2PPP/RNBQKB1R w KQkq - 0 4. What is the best move?
Prediction: [INVALID MOVE]
Actual: 

Input: White  move. Positi is rnbqkbnr/pp1p1ppp/4p3/8/3pP3/5N2/PPP2PPP/RNBQKB1R w KQkq - 0 4. What is the best move?
Prediction: [INVALID MOVE]
Actual: 

Input: White  move. Positi is rnbqkbnr/pp1p1ppp/4p3/8/3pP3/5N2/PPP2PPP/RNBQKB1R w KQkq - 0 4. What is the best move?
Prediction: [INVALID MOVE]
Actual: 

Sample Predictions:
Prediction 1: [INVALID MOVE]
Prediction 2: [INVALID MOVE]
Prediction 3: [INVALID MOVE]
Prediction 4: [INVALID MOVE]
Prediction 5: [INVALID MOVE]

Total predictions: 121


NameError: name 'best_eval_loss' is not defined